In [1]:
#Import required packages
from bs4 import BeautifulSoup
import requests
import os.path
import urllib.request
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, desc
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampNTZType

In [2]:
#os.makedirs("Yellow Taxi Trip Data")

In [3]:
#Get the data from the website
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

page = requests.get(url)

In [4]:
#Use BeautifulSoup for data scraping
soup = BeautifulSoup(page.content,"html.parser")
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]--><!--[if IE 7]><html class="no-js lt-ie9 lt-ie8 ie7"><![endif]--><!--[if IE 8]><html class="no-js lt-ie9"><![endif]--><!--[if gt IE 8]><!--><html class="no-js"><!--<![endif]--><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>TLC Trip Record Data - TLC</title>
<!--
					ls:begin[stylesheet]
				-->
<link href="/iwov-resources/fixed-layout/3-Row Simple.css" rel="stylesheet" type="text/css"/>
<!--
					ls:end[stylesheet]
				-->
<!--
					ls:begin[meta-keywords]
				-->
<meta content="" name="keywords"/>
<!--
					ls:end[meta-keywords]
				-->
<!--
					ls:begin[meta-description]
				-->
<meta content="" name="description"/>
<!--
					ls:end[meta-description]
				-->
<!--
					ls:begin[meta-vpath]
				-->
<meta content="" name="vpath"/>
<!--
					ls:end[meta-vpath]
				-->
<!--
					ls:begin[meta-page-locale-name]
				-->
<meta content="" name="page-locale-name"/>
<!--
		

In [5]:
#Get the links to download each parquet file
urls = []

#We have data from 2009-2023. This means there are 15 years of data for each month of the year 
for i in range(15):
    table_year = soup.find_all('table')[i] #find the table that is related to the year data

    yt = table_year.find_all('a',{"title":"Yellow Taxi Trip Records"}) #find the data related to Yellow Taxi Trip Records

    for month in range(len(yt)):
        #Get the link for each month of the year and add it to the urls list
        urls.append(table_year.find_all('a',{"title":"Yellow Taxi Trip Records"})[month].get("href"))


In [6]:
#Check if all links of data files are there 14years(from 2009-2022)*12months + 7months (2023) = 175
len(urls)

175

In [7]:
#Create a path for each data file 
filename_list = [] #all the name files will be added into that list
for url in urls:
    #cut the name of the url to get only the information about the date of the file
    test_name = url.rsplit('/', 1)[-1]
    final_name = test_name.split('.')[0]
    filename_list.append(final_name)

    filename = os.path.join('Yellow Taxi Trip Data', final_name) #create the filename path for each data file

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        print('Downloading: ' + filename)
        try:
            urllib.request.urlretrieve(url, filename)
        except Exception as inst:
            print(inst)
            print('Encountered unknown error. Continuing.')

In [8]:
# Initialize a SparkSession
spark = SparkSession.builder.appName('YellowTaxiTripData').getOrCreate()

23/10/23 07:02:59 WARN Utils: Your hostname, Reveccas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
23/10/23 07:02:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/23 07:03:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/23 07:03:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/23 07:03:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:
#Load into the dataframe the parquet file of July 2023
parquet_file_path = '/Users/reveccachristou/Desktop/Aptitude/Yellow Taxi Trip Data/yellow_tripdata_2023-01'
df1 = spark.read.parquet(parquet_file_path)

In [10]:
df1.select("total_amount").describe().toPandas()

,summary,total_amount
0,count,3066766
1,mean,27.02038310708492
2,stddev,22.163588952492184
3,min,-751.0
4,max,1169.4


In [11]:
#Find the record that has the maximum total amount value to check for outliers
filtered_df = df1.filter(df1['total_amount'] == 1169.4)
filtered_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-24 12:43:44|  2023-01-24 15:41:02|            1.0|       177.88|       4.0|                 N|         132|         265|           2|     1160.1|  0.0|    0.5|       0.

In [12]:
#Check how many negative-value records exist in the total_amount field
negative_amount_values_count = df1.filter(df1["total_amount"] < 0).count()
negative_amount_values_count

25204

In [13]:
negative_amount_values = df1.filter(df1["total_amount"] < 0)# .count()
negative_amount_values.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:28:29|  2023-01-01 00:31:03|            1.0|         0.42|       1.0|                 N|         233|         229|           4|       -5.1| -1.0|   -0.5|       0.

In [14]:
#Change the negative values to positive since this might be a wrong input
from pyspark.sql.functions import abs
df_with_positive = negative_amount_values.withColumn('total_amount', abs(negative_amount_values['total_amount']))
df_with_positive.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:28:29|  2023-01-01 00:31:03|            1.0|         0.42|       1.0|                 N|         233|         229|           4|       -5.1| -1.0|   -0.5|       0.

In [15]:
#Check if there are any null values. In this case, there are none.
df1.filter(df1.total_amount.isNull()).collect()

[]